### **Crawler - Reglas**

Comenzaremos trabajando con el siguiente website:

https://subslikescript.com/movies

En este video, vamos a trabajar con otro tipo de spider llamado "crawl spider". Hasta ahora cuando creamos la spider usando el comando "**`scrapy genspider`**" usamos la plantilla por defecto, que es la plantilla básica. La plantilla básica está hecha de una clase con diferentes propiedades como "**`name`**", "**`allowed_domains`**" "**`start_urls`**" y también tiene una función "**`parse()`**". Pero también se puede trabajar con otro tipo de plantillas. Así que abre la terminal y escribe este comando: **`scrapy genspider -l`**, luego presiona enter y verás todas las plantillas disponibles.

<center><img src="https://i.postimg.cc/KYQHq7Vy/ws-134.png"></center>

Ahora vamos a crear un nuevo spider utilizando una crawl template. Así que escribimos "**`scrapy genspider -t crawl`**" y luego escribimos el nombre de la spider que queremos crear. En este caso, voy a llamarlo "**`transcripts`**". A continuación, vamos a ir a la página web para copiar el enlace y luego vamos a pegarlo.

<center><img src="https://i.postimg.cc/B66wdL9P/ws-135.png"></center>

Pero ahora tenemos una nueva propiedad llamada "**`rules`**", que es una tupla. Una tupla funciona como una lista, pero son inmutables, lo que significa que una vez que la configuras, no puedes cambiar lo que hay dentro de ella. La tupla "rules" debe contener al menos un objeto regla. El objeto "rules" se utiliza para indicar a la crawl spider uno de los enlaces que quiere seguir (follow) en el sitio web que estamos scrapeando. Por defecto, ya tenemos una regla configurada. Esta regla tiene tres argumentos. El primer argumento es "**`allow`**". El segundo es el método "**`callback`**", que **`se establece como un string`**. A diferencia de lo que hemos visto en la plantilla básica, el método callback en la crawl spider debe definirse como un string. El último argumento es el argumento "**`follow`**". Es un argumento booleano y se establece a "**`true`**" y enviará una solicitud (request) a los enlaces (links) extraídos. Finalmente, tenemos el objeto "**`LinkExtractor`**". Sirve para especificar los enlaces que queremos extraer o no. Para definir los enlaces que queremos extraer, utilizamos el argumento "allow". En este caso, se indica una expresión regular: r'Items/'. Esto significa que si el enlace contiene la palabra "items" seguida y luego parsear la respuesta en la función "parse_item()". Lo contrario de este argumento es el argumento "**`deny`**". En este caso, si el enlace contiene la palabra "item", entonces será seguido. También existe el argumento "**`restrict_xpaths`**". Este restringe la región que queremos que la crawl spider extraiga los enlaces. 

<center><img src="https://i.postimg.cc/bYGqndDv/ws-136.png"></center>

Vamos a inspeccionar el websit con el cual trabajremos. El siguiente match va a coincidir con la caja que contiene todas las películas listadas:

<center><img src="https://i.postimg.cc/76WQDHws/ws-137.png"></center>
<center><img src="https://i.postimg.cc/nr9ShyLH/ws-138.png"></center>

Va a listar cada uno de los enlaces de las peliculas:

<center><img src="https://i.postimg.cc/qgfbh1t2/ws-139.png"></center>

Por lo tanto, utilizamos por el momento el siguiente código para verificar que está todo correcto:

In [ ]:
import scrapy
from scrapy.linkextractors import LinkExtractor
from scrapy.spiders import CrawlSpider, Rule


class TranscriptsSpider(CrawlSpider):
    name = 'transcripts'
    allowed_domains = ['subslikescript.com']
    start_urls = ['https://subslikescript.com/movies']

    # Establecer reglas para el crawler
    # Así que seguiremos sólo los enlaces de los elementos que tengan este XPath (restrict_xpaths).
    # Además, como se puede ver, no he añadido el "/@href" (//ul[@class='scripts-list']/a/@href) esto es porque el 
    # objeto "LinkExtractor" buscará automáticamente el atributo "href", por lo que tenemos que omitirlo.
    rules = (
        Rule(LinkExtractor(restrict_xpaths=("//ul[@class='scripts-list']/a")), callback='parse_item', follow=True),
    )

    def parse_item(self, response):
        print(response.url)

Si ejecutamos en el terminal:

<center><img src="https://i.postimg.cc/tgq1Pccy/ws-140.png"></center>

Nos devuelve todos los enlaces de la PRIMERA PÁGINA:

<center><img src="https://i.postimg.cc/43WY15Cc/ws-141.png"></center>

Vamos a trabajar con el enlace:

https://subslikescript.com/movies_letter-X

Lo que buscamos es extraer el **`TITULO`**, **`RESEÑA`** y la **`TRANSCRIPCIÓN`** de cada pelicula que comience con X que se encuentre en la **`PRIMERA PÁGINA`**:

<center><img src="https://i.postimg.cc/Dz5cbH23/ws-142.png"></center>

Y vamos a escoger cualquier pelicula para inspeccionar su página:

<center><img src="https://i.postimg.cc/zBBFw7Bb/ws-143.png"></center>

Buscamos la caja que encierre todo el contenido de la página:

<center><img src="https://i.postimg.cc/BbP5kv2J/ws-144.png"></center>

Encontramos su titulo:

<center><img src="https://i.postimg.cc/mZP0BNwg/ws-145.png"></center>

su introducción:

<center><img src="https://i.postimg.cc/nM8KJt1B/ws-146.png"></center>

Y finalmente, su transcripción:

<center><img src="https://i.postimg.cc/3RcCtG4L/ws-147.png"></center>

Este es el código que utilizamos:

In [ ]:
import scrapy
from scrapy.linkextractors import LinkExtractor
from scrapy.spiders import CrawlSpider, Rule


class TranscriptsSpider(CrawlSpider):
    name = 'transcripts'
    allowed_domains = ['subslikescript.com']
    # Utilizamos el enlace para las peliculas con X (son pocas)
    start_urls = ['https://subslikescript.com/movies_letter-X']

    # Establecer reglas para el crawler
    # Así que seguiremos sólo los enlaces de los elementos que tengan este XPath (restrict_xpaths).
    # Además, como se puede ver, no he añadido el "/@href" (//ul[@class='scripts-list']/a/@href) esto es porque el 
    # objeto "LinkExtractor" buscará automáticamente el atributo "href", por lo que tenemos que omitirlo.
    rules = (
        Rule(LinkExtractor(restrict_xpaths=("//ul[@class='scripts-list']/a")), callback='parse_item', follow=True),
    )

    def parse_item(self, response):
        # Obtener la caja del article que contiene los datos que queremos (title, plot, etc)
        article = response.xpath("//article[@class='main-article']")

        # Extraer los datos que queremos y luego devolverlos
        yield {
            'title': article.xpath("./h1/text()").get(),
            'plot': article.xpath("./p/text()").get(),
            'transcript': article.xpath("./div[@class='full-script']/text()").getall(),
            'url': response.url,
        }


Si ejecutamos en el terminal:

<center><img src="https://i.postimg.cc/tJr3sxyd/ws-148.png"></center>

Nos devuelve:

<center><img src="https://i.postimg.cc/tCMdyWBp/ws-149.png"></center>

Ahora, si queremos extraerlo en un archivo **`CSV`**:

<center><img src="https://i.postimg.cc/wxFcfSJt/ws-150.png"></center>
<center><img src="https://i.postimg.cc/VL693DBK/ws-151.png"></center>